In [1]:
import os
import json
import pandas as pd
import numpy as np

current_directory = os.getcwd()
path_components = current_directory.split(os.path.sep)
results_dir = os.path.sep.join(path_components[:-3])


In [2]:
def find_largest_iteration(folder_path):
    subdirectories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    iteration_numbers = [int(d.split('_')[1]) for d in subdirectories if d.startswith('iter_')]
    if not iteration_numbers:
        return None
    largest_iteration_folder = os.path.join(folder_path, f'iter_{max(iteration_numbers)}')
    return largest_iteration_folder

def find_loss_csv(folder_path):
    largest_iteration_folder = find_largest_iteration(os.path.join(folder_path,'iterations'))
    if largest_iteration_folder:
        loss_csv_path = os.path.join(largest_iteration_folder, 'loss.csv')
        if os.path.isfile(loss_csv_path):
            return loss_csv_path
    return None

def get_last_iteration_losses(loss_csv_path):
    df = pd.read_csv(loss_csv_path)
    last_row = df.iloc[-1]
    losses = last_row[list(df.columns)]
    return losses

def read_results_json(folder_path):
    results_json_path = os.path.join(folder_path, 'results_values.json')
    if os.path.isfile(results_json_path):
        with open(results_json_path, 'r') as json_file:
            results_data = json.load(json_file)
        return results_data
    else:
        return None

def get_results_by_sim(folder_path):
    loss_csv_path = find_loss_csv(folder_path)
    if loss_csv_path:
        results = get_last_iteration_losses(loss_csv_path)
        results_data = read_results_json(folder_path)
        if results_data:
            for name in results_data:
                results[name] = float(results_data[name])
            return results
    return None

In [3]:
molecule = 'born_ion'
results_dir = os.path.join(results_dir,'.results',molecule)
print(results_dir)

/home/martinachondo/XPBE/.results/born_ion


In [6]:
folders = ['.architecture']
data = dict()
for folder in folders:
    folder_path = os.path.join(results_dir,folder) if folder!= '' else results_dir
    sims = os.listdir(folder_path)
    for sim in sims:
        sim_path = os.path.join(folder_path,sim)
        result = get_results_by_sim(sim_path)
        data[sim] = result

data_filtered = {sim: result for sim, result in data.items() if result is not None}
df = pd.DataFrame.from_dict(data_filtered, orient='index')

filter_list = ["Gsolv_value",
                "Loss_XPINN",
                "Loss_NN1",
                "Loss_NN2",
                "Loss_Val_NN1",
                "Loss_Val_NN2",
                "Loss_continuity_u",
                "Loss_continuity_du",
                "Loss_Residual_R1",
                "Loss_Residual_R2",
                "Loss_Boundary_D2",
                "Loss_Data_K2",
                "L2_analytic"]
df_filtered = df.filter(filter_list)

excel_file_path = 'results_temp.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='DF_complete', index=True)
    df_filtered.to_excel(writer, sheet_name='DF_Filtered', index=True)

In [7]:
df_filtered

,Gsolv_value,Loss_XPINN,Loss_NN1,Loss_NN2,Loss_Val_NN1,Loss_Val_NN2,Loss_continuity_u,Loss_continuity_du,Loss_Residual_R1,Loss_Residual_R2,Loss_Boundary_D2,Loss_Data_K2,L2_analytic
B17_FO_H4_N200_E_FA_WF_ModMLP,-165.414,0.011760,0.000218,0.000177,0.000400,0.000335,6.856000e-07,1.681000e-04,4.965000e-05,8.176000e-06,4.316000e-08,8.289000e-08,48.74
B13_H4_N200,-165.255,0.006527,0.005018,0.005018,0.004926,0.004926,7.650000e-08,5.018000e-03,1.259000e-14,6.791000e-08,2.417000e-08,4.154000e-08,42.43
B6_FO_H2_N200_E_FA,-165.388,0.011800,0.001532,0.001040,0.001662,0.000930,8.895000e-07,1.021000e-03,5.096000e-04,1.789000e-05,1.972000e-07,2.538000e-07,52.29
B7_FO_H6_N200_E_FA,-165.311,0.001049,0.000071,0.000020,0.000268,0.000134,3.433000e-08,9.089000e-07,7.054000e-05,1.878000e-05,1.761000e-07,8.551000e-08,47.66
B11_FO_H4_N200_E,-164.610,0.002805,0.000241,0.000065,0.000584,0.000307,1.033000e-07,2.031000e-05,2.207000e-04,4.419000e-05,4.364000e-08,1.726000e-08,62.28
B8_FO_H4_N100_E_FA,-165.336,0.002075,0.000448,0.000008,0.000404,0.000008,3.115000e-08,6.624000e-06,4.414000e-04,1.539000e-06,3.338000e-09,5.685000e-09,47.63
B12_FO_H4_N200,-166.418,0.006052,0.000581,0.000920,0.000826,0.001288,1.376000e-08,5.751000e-04,5.645000e-06,3.446000e-04,5.104000e-09,4.875000e-09,21.75
B4_FO_H4_N200_E_FA,-164.719,0.001927,0.000440,0.000030,0.000585,0.000037,1.203000e-07,2.632000e-05,4.135000e-04,3.643000e-06,1.601000e-08,1.155000e-08,56.62
B10_FO_H4_N200_FA,-166.509,0.004567,0.000460,0.000657,0.000606,0.000710,8.471000e-09,4.105000e-04,4.934000e-05,2.469000e-04,4.558000e-09,4.709000e-09,23.58
B16_FO_H4_N200_E_FA_WF,-165.759,0.011790,0.000663,0.000812,0.001041,0.001398,2.442000e-06,5.989000e-04,6.160000e-05,2.070000e-04,1.826000e-06,1.944000e-06,53.52
